In [2]:
!pip install tflearn


     |████████████████████████████████| 107 kB 20.3 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=29ea32c82b8737dacfeddd5a8a842c179cbc6a04c02abd8def0134203ef6cb32
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn


In [3]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

Instructions for updating:
non-resource variables are not supported in the long term
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
#Processing the Intents
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [7]:
words = []
classes = []
documents = []
ignore_words = ['?']
#Looping through the Intents to Convert them to words, classes, documents and ignore_words.......
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            

In [8]:
print("Stemming, Lowering and Removing Duplicates")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates
36 documents
12 classes ['Food', 'Later', 'bar', 'booking', 'goodbye', 'greeting', 'hours', 'opentoday', 'payments', 'rental', 'thanks', 'today']
57 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'avail', 'bar', 'book', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'facil', 'food', 'good', 'goodby', 'hello', 'help', 'hey', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'of', 'off', 'on', 'op', 'pub', 'room', 'see', 'serv', 'suit', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'typ', 'veget', 'we', 'what', 'when', 'work', 'yo', 'you']


In [9]:
#Creating the Data for our Model
training = []
output = []
#Creating an List (Empty) for Output
output_empty = [0] * len(classes)

#Creating Traning Set, Bag of Words for our Model
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [10]:
#Shuffling Randomly and Converting into Numpy Array for Faster Processing
random.shuffle(training)
training = np.array(training)

#Creating Train and Test Lists
train_x = list(training[:,0])
train_y = list(training[:,1])
#Building Neural Network for Out Chatbot
#Resetting graph data
tf.compat.v1.reset_default_graph()

In [11]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training model")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training model


In [12]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [13]:
#Training the Model
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model")
model.save('model.tflearn')

Training Step: 4999  | total loss: 0.01598 | time: 0.025s
| Adam | epoch: 1000 | loss: 0.01598 - acc: 1.0000 -- iter: 32/36
Training Step: 5000  | total loss: 0.01540 | time: 0.033s
| Adam | epoch: 1000 | loss: 0.01540 - acc: 1.0000 -- iter: 36/36
--
Saving the Model.......
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [14]:
#Saved Pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [15]:
#Loading Pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('intents.json') as json_data:
    intents = json.load(json_data)
    
#Loading the saved Model
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......
INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [16]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
#ERROR_THRESHOLD = 0.25

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [17]:
class CleanExit(object):
     def __enter__(self):
             return self
     def __exit__(self, exc_type, exc_value, exc_tb):
             if exc_type is KeyboardInterrupt:
                     return True
             return exc_type is None

In [18]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

You- hi
Hello, thanks for visiting Saanj Palace
You- Food available?
Hi there, how can I help?
You- Is food services available?
Yes, We offer food for various tastes, including Vegetarian, Sea food, Italian, Mexiacn, Indian and so 
You- Food services?
Yes, We offer food for various tastes, including Vegetarian, Sea food, Italian, Mexiacn, Indian and so 
You- Bar facility?
Yes it is available, we avail two types, The Elite(11am to 3 pm), The Sassy(8pm to 2 am)


KeyboardInterrupt: ignored